# 1. 读取文件

In [1]:
import pandas as pd

In [2]:
df_total = pd.read_excel('./2.WEEK2—case题目0428.xlsx',
                         sheet_name='总表',
                         header=0,
                         usecols='A:B',
                         dtype=str).fillna('')

df_total

,SKU,仓库
0,WS02863,美东仓库
1,WS02632,美西仓库
2,OFA294AEB,美西仓库
3,OFA583AFB,美西仓库
4,AIHAAA00,美东仓库
...,...,...
28426,GS70140,美南仓库
28427,GS71310,美南仓库
28428,GS68880,美南仓库
28429,GS07168,美南仓库


In [3]:
df_category_1 = pd.read_excel('./2.WEEK2—case题目0428.xlsx',
                              sheet_name='附2_美国品类',
                              header=0,
                              usecols='A',
                              dtype=str).fillna('')

df_category_1

,品类
0,WS
1,WS
2,OFA
3,OFA
4,AIH
...,...
28561,GS
28562,GS
28563,GS
28564,GS


In [4]:
df_category_2 = pd.read_excel('./2.WEEK2—case题目0428.xlsx',
                              sheet_name='附2_美国品类',
                              header=1,
                              usecols='D:E',
                              dtype=str).fillna('')[:14]

df_category_2

,SKU,品类
0,BFOPTG5519,BFOPT
1,BFOPTF5519,BFOPT
2,BR583AB,BMR
3,BR5Z000,BMR
4,BRGW41Z,BMR
5,BR06210K,BMR
6,BRDGW51,BMR
7,BR0W014,BMR
8,BR91399,BMR
9,BR9263A,BMR


In [5]:
# 去重
df_category_1 = df_category_1.drop_duplicates(ignore_index=True)
# 排序
df_category_1 = df_category_1.sort_values(by=['品类'], ascending=[True]).reset_index(drop=True)

df_category_1

,品类
0,AAS
1,ABS
2,ACB
3,ACC
4,ACH
...,...
296,WP
297,WR
298,WS
299,WT


# 2. 先处理特殊SKU

In [6]:
df_total = pd.merge(left=df_total,
                   right=df_category_2,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna('')

df_total.columns = df_total.columns.tolist()[:-1] + ['特殊品类']

df_total

,SKU,仓库,特殊品类
0,WS02863,美东仓库,
1,WS02632,美西仓库,
2,OFA294AEB,美西仓库,
3,OFA583AFB,美西仓库,
4,AIHAAA00,美东仓库,
...,...,...,...
28426,GS70140,美南仓库,
28427,GS71310,美南仓库,
28428,GS68880,美南仓库,
28429,GS07168,美南仓库,


# 3. 处理普通SKU
## 3.1 方法一

In [7]:
df_total_1 = df_total.copy()

df_total_1

,SKU,仓库,特殊品类
0,WS02863,美东仓库,
1,WS02632,美西仓库,
2,OFA294AEB,美西仓库,
3,OFA583AFB,美西仓库,
4,AIHAAA00,美东仓库,
...,...,...,...
28426,GS70140,美南仓库,
28427,GS71310,美南仓库,
28428,GS68880,美南仓库,
28429,GS07168,美南仓库,


In [8]:
# SKU首两个、三个或四个字符为该SKU对应的品类，依次匹配
df_total_1['head_2'] = [sku[:2] for sku in df_total_1['SKU'].tolist()]
df_total_1['head_3'] = [sku[:3] for sku in df_total_1['SKU'].tolist()]
df_total_1['head_4'] = [sku[:4] for sku in df_total_1['SKU'].tolist()]

df_total_1

,SKU,仓库,特殊品类,head_2,head_3,head_4
0,WS02863,美东仓库,,WS,WS0,WS02
1,WS02632,美西仓库,,WS,WS0,WS02
2,OFA294AEB,美西仓库,,OF,OFA,OFA2
3,OFA583AFB,美西仓库,,OF,OFA,OFA5
4,AIHAAA00,美东仓库,,AI,AIH,AIHA
...,...,...,...,...,...,...
28426,GS70140,美南仓库,,GS,GS7,GS70
28427,GS71310,美南仓库,,GS,GS7,GS71
28428,GS68880,美南仓库,,GS,GS6,GS68
28429,GS07168,美南仓库,,GS,GS0,GS07


In [9]:
df_total_1 = pd.merge(left=df_total_1,
                      right=df_category_1,
                      how='left',
                      left_on='head_2',
                      right_on='品类').fillna('')

df_total_1.columns = df_total_1.columns.tolist()[:-1] + ['品类_2']

df_total_1

,SKU,仓库,特殊品类,head_2,head_3,head_4,品类_2
0,WS02863,美东仓库,,WS,WS0,WS02,WS
1,WS02632,美西仓库,,WS,WS0,WS02,WS
2,OFA294AEB,美西仓库,,OF,OFA,OFA2,
3,OFA583AFB,美西仓库,,OF,OFA,OFA5,
4,AIHAAA00,美东仓库,,AI,AIH,AIHA,
...,...,...,...,...,...,...,...
28426,GS70140,美南仓库,,GS,GS7,GS70,GS
28427,GS71310,美南仓库,,GS,GS7,GS71,GS
28428,GS68880,美南仓库,,GS,GS6,GS68,GS
28429,GS07168,美南仓库,,GS,GS0,GS07,GS


In [10]:
df_total_1 = pd.merge(left=df_total_1,
                      right=df_category_1,
                      how='left',
                      left_on='head_3',
                      right_on='品类').fillna('')

df_total_1.columns = df_total_1.columns.tolist()[:-1] + ['品类_3']

df_total_1

,SKU,仓库,特殊品类,head_2,head_3,head_4,品类_2,品类_3
0,WS02863,美东仓库,,WS,WS0,WS02,WS,
1,WS02632,美西仓库,,WS,WS0,WS02,WS,
2,OFA294AEB,美西仓库,,OF,OFA,OFA2,,OFA
3,OFA583AFB,美西仓库,,OF,OFA,OFA5,,OFA
4,AIHAAA00,美东仓库,,AI,AIH,AIHA,,AIH
...,...,...,...,...,...,...,...,...
28426,GS70140,美南仓库,,GS,GS7,GS70,GS,
28427,GS71310,美南仓库,,GS,GS7,GS71,GS,
28428,GS68880,美南仓库,,GS,GS6,GS68,GS,
28429,GS07168,美南仓库,,GS,GS0,GS07,GS,


In [11]:
df_total_1 = pd.merge(left=df_total_1,
                      right=df_category_1,
                      how='left',
                      left_on='head_4',
                      right_on='品类').fillna('')

df_total_1.columns = df_total_1.columns.tolist()[:-1] + ['品类_4']

df_total_1

,SKU,仓库,特殊品类,head_2,head_3,head_4,品类_2,品类_3,品类_4
0,WS02863,美东仓库,,WS,WS0,WS02,WS,,
1,WS02632,美西仓库,,WS,WS0,WS02,WS,,
2,OFA294AEB,美西仓库,,OF,OFA,OFA2,,OFA,
3,OFA583AFB,美西仓库,,OF,OFA,OFA5,,OFA,
4,AIHAAA00,美东仓库,,AI,AIH,AIHA,,AIH,
...,...,...,...,...,...,...,...,...,...
28426,GS70140,美南仓库,,GS,GS7,GS70,GS,,
28427,GS71310,美南仓库,,GS,GS7,GS71,GS,,
28428,GS68880,美南仓库,,GS,GS6,GS68,GS,,
28429,GS07168,美南仓库,,GS,GS0,GS07,GS,,


In [12]:
df_total_1['品类'] = [df_total_1.loc[i, '特殊品类'] if df_total_1.loc[i, '特殊品类'] != '' else df_total_1.loc[i, '品类_4'] if df_total_1.loc[i, '品类_4'] != '' else df_total_1.loc[i, '品类_3'] if df_total_1.loc[i, '品类_3'] != '' else df_total_1.loc[i, '品类_2'] for i in range(len(df_total_1))]

df_total_1

,SKU,仓库,特殊品类,head_2,head_3,head_4,品类_2,品类_3,品类_4,品类
0,WS02863,美东仓库,,WS,WS0,WS02,WS,,,WS
1,WS02632,美西仓库,,WS,WS0,WS02,WS,,,WS
2,OFA294AEB,美西仓库,,OF,OFA,OFA2,,OFA,,OFA
3,OFA583AFB,美西仓库,,OF,OFA,OFA5,,OFA,,OFA
4,AIHAAA00,美东仓库,,AI,AIH,AIHA,,AIH,,AIH
...,...,...,...,...,...,...,...,...,...,...
28426,GS70140,美南仓库,,GS,GS7,GS70,GS,,,GS
28427,GS71310,美南仓库,,GS,GS7,GS71,GS,,,GS
28428,GS68880,美南仓库,,GS,GS6,GS68,GS,,,GS
28429,GS07168,美南仓库,,GS,GS0,GS07,GS,,,GS


In [13]:
df_total_1 = df_total_1[['SKU', '仓库', '品类']]

df_total_1.to_excel('./answer_1_1.xlsx', index=False)

df_total_1

,SKU,仓库,品类
0,WS02863,美东仓库,WS
1,WS02632,美西仓库,WS
2,OFA294AEB,美西仓库,OFA
3,OFA583AFB,美西仓库,OFA
4,AIHAAA00,美东仓库,AIH
...,...,...,...
28426,GS70140,美南仓库,GS
28427,GS71310,美南仓库,GS
28428,GS68880,美南仓库,GS
28429,GS07168,美南仓库,GS


In [14]:
# 方法一的另一种实现形式， from Miang

# for i in range(len(df_zongbiao)):
#     if df_zongbiao.loc[i,'SKU'][0:4] in df_pinlei['品类'].to_list():
#         df_zongbiao.loc[i,'品类'] = df_zongbiao.loc[i,'SKU'][0:4]
#     elif df_zongbiao.loc[i,'SKU'][0:3] in df_pinlei['品类'].to_list():
#         df_zongbiao.loc[i,'品类'] = df_zongbiao.loc[i,'SKU'][0:3]
#     elif df_zongbiao.loc[i,'SKU'][0:2] in df_pinlei['品类'].to_list():
#         df_zongbiao.loc[i,'品类'] = df_zongbiao.loc[i,'SKU'][0:2]
#     elif df_zongbiao.loc[i,'SKU'] in df_pinlei['特殊SKU对应品类'].to_list():
#         df_zongbiao.loc[i,'品类'] = [df_pinlei["特殊SKU对应品类"].to_list().index(df_zongbiao.loc[i,'SKU'])]
#     else:
#         pass

## 3.2 方法二

In [15]:
df_total_2 = df_total.copy()

df_total_2

,SKU,仓库,特殊品类
0,WS02863,美东仓库,
1,WS02632,美西仓库,
2,OFA294AEB,美西仓库,
3,OFA583AFB,美西仓库,
4,AIHAAA00,美东仓库,
...,...,...,...
28426,GS70140,美南仓库,
28427,GS71310,美南仓库,
28428,GS68880,美南仓库,
28429,GS07168,美南仓库,


In [16]:
import re

In [17]:
# 从SKU的构成的一般形式下手，品类缩写 + 以数字开头的5位OE号
# 检查结果发现这个方法在这个题目里不是很行，因为早期的SKU并不规范
df_total_2['疑似品类'] = [re.match(r'^[a-zA-Z]{2,4}', sku).group() for sku in df_total_2['SKU'].tolist()]

df_total_2

,SKU,仓库,特殊品类,疑似品类
0,WS02863,美东仓库,,WS
1,WS02632,美西仓库,,WS
2,OFA294AEB,美西仓库,,OFA
3,OFA583AFB,美西仓库,,OFA
4,AIHAAA00,美东仓库,,AIHA
...,...,...,...,...
28426,GS70140,美南仓库,,GS
28427,GS71310,美南仓库,,GS
28428,GS68880,美南仓库,,GS
28429,GS07168,美南仓库,,GS


In [18]:
df_total_2['品类'] = [df_total_2.loc[i, '特殊品类'] if df_total_2.loc[i, '特殊品类'] != '' else df_total_2.loc[i, '疑似品类'] for i in range(len(df_total_2))]

df_total_2

,SKU,仓库,特殊品类,疑似品类,品类
0,WS02863,美东仓库,,WS,WS
1,WS02632,美西仓库,,WS,WS
2,OFA294AEB,美西仓库,,OFA,OFA
3,OFA583AFB,美西仓库,,OFA,OFA
4,AIHAAA00,美东仓库,,AIHA,AIHA
...,...,...,...,...,...
28426,GS70140,美南仓库,,GS,GS
28427,GS71310,美南仓库,,GS,GS
28428,GS68880,美南仓库,,GS,GS
28429,GS07168,美南仓库,,GS,GS


In [19]:
df_total_2 = df_total_2[['SKU', '仓库', '品类']]

df_total_2.to_excel('./answer_1_2.xlsx', index=False)

df_total_2

,SKU,仓库,品类
0,WS02863,美东仓库,WS
1,WS02632,美西仓库,WS
2,OFA294AEB,美西仓库,OFA
3,OFA583AFB,美西仓库,OFA
4,AIHAAA00,美东仓库,AIHA
...,...,...,...
28426,GS70140,美南仓库,GS
28427,GS71310,美南仓库,GS
28428,GS68880,美南仓库,GS
28429,GS07168,美南仓库,GS
